In [ ]:
import gmsh

import sys
%matplotlib inline
import matplotlib.pyplot as plt
from sympy import solve, symbols
import cmath
from math import pi, sin, cos, asin, log10, log2
import numpy as np
import pandas as pd
import altair as alt

from firedrake import *
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower


In [ ]:
mesh = Mesh('horn.msh')
# r = 2
# mesh = UnitSquareMesh(2 ** r, 2 ** r, quadrilateral=quadrilateral)



In [ ]:
V = FunctionSpace(mesh, "CG", 1)
VB = VectorFunctionSpace(mesh, "CG", 1)

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
fct = Function(V)
x, y = SpatialCoordinate(mesh)
normal = FacetNormal(mesh)

In [ ]:
g_surf = 1
g_inlet = 2
g_outlet = 3
# length of the boundaries
l_inlet = sqrt(assemble(dot(normal, normal)*ds(g_inlet)))
l_outlet = sqrt(assemble(dot(normal, normal)*ds(g_outlet)))

# speed of sound in air
c_air = 343.344
# 
rho_air = 1.20458
# incoming wave amplitude
p_A_in = 1
# outgoing wave amplitude
p_B_out = 1
# radius of outer circle
p_R_omega = 1
# 2 case, plannar and cylindrical
# plannar
p_kappa = 2
p_r = 1
# cyclindrical
# kappa = 1
# r = distance to symmetric axis

def helmotz_complex(freq):
    omega = (2*pi*freq)
    omega2 = omega**2
    coeff_laplace = 1/rho_air
    coeff_pressure = -1/c_air/c_air/rho_air
    coeff_in = 1/c_air/rho_air
    coeff_out = 1/rho_air/2
    # print('coeff laplacian {} pressure {} neumann IN {} neumann OUT {}'.format(coeff_laplace, coeff_pressure, coeff_in, coeff_out))
    # left hand side
    a_prod = (coeff_laplace*inner(grad(u), grad(v)) + coeff_pressure*omega2*inner(u, v)) * dx
    # right hand side
    a_in1 =            -1j*coeff_in*omega*(1+dot(normal, normal)/l_inlet)
    a_in2 = (          -1j*coeff_in*omega)*inner(u, v)*ds(g_inlet)
    a_out = (-coeff_out-1j*coeff_in*omega)*inner(u, v)*ds(g_outlet)
    a = a_prod - a_in2 - a_out
    fct.interpolate(x)
    L = inner(1j*(1+dot(normal, normal)), v)*ds(g_inlet)
    # solve
    assemble(a)
    assemble(L)
    sol = Function(V)
    solve(a == L, sol, solver_parameters={'ksp_type': 'cg'})
    return sol

In [ ]:
sol900 = helmotz_complex(900)

In [ ]:
plot_contour(900, sol900)

In [ ]:
out = File("horn1.pvd")
out.write(sol900)

In [ ]:
def plot_contour(freq, sol):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,6))
    #levels= np.linspace(-60, 10, 51)
    levels = 21
    ax1.set_aspect('equal')
    # ax1.legend()
    colors= tripcolor(sol, axes=ax1)
    fig.colorbar(colors, ax=ax1)
    ax2.set_aspect('equal')
    contours = tricontour(sol, axes=ax2, levels=levels, cmap="inferno")
    fig.colorbar(contours)
    plt.title('{}Hz'.format(freq))
    plt.show()

In [ ]:
results = {}
for hz in (50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 1000):
    sol_hz = helmotz_complex(hz)
    results[hz] = sol_hz

In [ ]:
for f_hz, res in results.items():
    plot_contour(f_hz, res)

In [ ]:
j = {}
for freq, sol in results.items():
    j[freq] = abs(abs(assemble(sol*ds(g_inlet)))/l_inlet-1)

In [ ]:
j

In [ ]:
freq = np.linspace(50, 1000, 100)
results = []
reflections = []
for f_hz in freq:
    s_hz = helmotz_complex(f_hz)
    results.append(s_hz)
    reflections.append(abs(abs(assemble((s_hz*ds(g_inlet))))/l_inlet-1))

In [ ]:
alt.Chart(pd.DataFrame({'Freq': freq, 'Reflection': reflections})).mark_line(
).encode(
    x=alt.X('Freq', title='Freq (Hz)'), #scale=alt.Scale(type="lin", base=10, nice=False, domain=[20, 1000])), 
    y=alt.Y('Reflection', scale=alt.Scale(type="log", base=10)),
)

In [ ]:
def angle(theta):
    if theta == 0:
        return 'On Axis'
    return '{}°'.format(int(theta))

r = 0.999
dfs = []
for theta in np.linspace(0, 180, 19):
    dbs = []
    theta_rad = theta*pi/180
    p_x = r*cos(theta_rad)
    p_y = r*sin(theta_rad)
    for fr, sol in zip(freq, results):
        p_p = sol.at(p_x, p_y, dont_raise=True)
        if p is not None:
            p_db = 105+20*log10(abs(p_p))
            # print('{:3.0f} {:+0.2f} {:+0.2f} {}'.format(theta, p_x, p_y, p_db))
            dbs.append(p_db)
        else:
            print('{} {} {} ERROR'.format(theta, p_x, p_y))
    if theta == 0:
        dfs.append(pd.DataFrame({"Freq": freq, angle(theta): dbs}))
    else:
        angle_p = angle(theta)
        angle_m = '-{}'.format(angle_p)
        dfs.append(pd.DataFrame({angle_p: dbs}))
        if theta != 180:
            dfs.append(pd.DataFrame({angle_m: dbs}))

df = pd.concat(dfs, axis=1)        

In [ ]:
dfs = df.reset_index().melt(id_vars='Freq', var_name='Measurements', value_name='dB').loc[lambda df: df["Measurements"] != "index"]
dfs.head()

In [ ]:
alt.Chart(dfs).mark_line(
).transform_filter(
    alt.FieldOneOfPredicate(
        field="Measurements",
        oneOf=[
            "On Axis",
            "10°",
            "20°",
            "30°",
        ],),
).encode(
    x=alt.X('Freq:Q', scale=alt.Scale(type="log", base=10, nice=False, domain=[20, 20000])),
    y=alt.Y('dB:Q'),
    color=alt.Color('Measurements')
)

In [ ]:
import sys,os,os.path
sys.path.append(os.path.expanduser('/home/pierre/src/spinorama/src'))
from spinorama.graph import graph_contour, contour_params_default

In [ ]:
graph_contour(df, contour_params_default)